## Importing Libraries


In [98]:
# Standard libraries
import os
from datetime import datetime, timedelta
from io import StringIO

# Third-party libraries
import requests
import pandas as pd
import numpy as np
import statsmodels.api as sm
import plotly.express as px

## Fetching Data from BMRS Elexon Data
1st Oct 2022 to 31st Dec 2024

In [99]:
def fetch_in_chunks(start_date, end_date, chunk_days, fetch_function):
    current_date = start_date
    while current_date <= end_date:
        next_date = min(current_date + timedelta(days=chunk_days - 1), end_date)
        fetch_function(current_date.strftime('%Y-%m-%d'), next_date.strftime('%Y-%m-%d'))
        current_date = next_date + timedelta(days=1)

def fetch_csv_data(api_url, params, save_file, sort_by=None, parse_dates=None, headers=None):
    response = requests.get(api_url, params=params, headers=headers or {})
    if response.status_code != 200:
        print(f"[ERROR] Failed fetch: {response.status_code} | URL: {api_url}")
        return

    df_new = pd.read_csv(StringIO(response.text), parse_dates=parse_dates)
    if df_new.empty:
        print("[INFO] No new data returned.")
        return

    if os.path.exists(save_file):
        df_existing = pd.read_csv(save_file, parse_dates=parse_dates)
        df_combined = pd.concat([df_existing, df_new]).drop_duplicates()
    else:
        df_combined = df_new

    if sort_by:
        df_combined = df_combined.sort_values(by=sort_by)

    df_combined.to_csv(save_file, index=False)
    print(f"[SUCCESS] Data saved to {save_file}")

Wind and Solar Day Ahead Forecast Data

Wind and Solar Actual Generation Data

Wind Generation Forecast Data 

Demand Forecast Data 

Temperature Data

Great Britain Electricity Market Index Data

In [100]:
FETCH_CONFIGS = {
    "wind_solar_dayahead_forecast": {
        "api_url": "https://data.elexon.co.uk/bmrs/api/v1/forecast/generation/wind-and-solar/day-ahead",
        "params": lambda start, end: {
            "from": start,
            "to": end,
            "settlementPeriodFrom": 48,
            "settlementPeriodTo": 0,
            "processType": "intraday process",
            "format": "csv"
        },
        "save_file": "data/wind_solar_dayahead_forecast.csv",
        "chunk_days": 7
    },
    "wind_solar_actual_generation": {
        "api_url": "https://data.elexon.co.uk/bmrs/api/v1/generation/actual/per-type/wind-and-solar",
        "params": lambda start, end: {
            "from": start,
            "to": end,
            "settlementPeriodFrom": 48,
            "settlementPeriodTo": 1,
            "format": "csv"
        },
        "save_file": "data/wind_solar_actual_generation.csv",
        "chunk_days": 7
    },
    "wind_forecast_winfor": {
        "api_url": "https://data.elexon.co.uk/bmrs/api/v1/datasets/WINDFOR",
        "params": lambda start, end: {
            "publishDateTimeFrom": start,
            "publishDateTimeTo": end,
            "format": "csv"
        },
        "save_file": "data/wind_generation_forecast.csv",
        "chunk_days": 7
    },
    "dayahead_demand_forecast": {
        "api_url": "https://data.elexon.co.uk/bmrs/api/v1/forecast/demand/day-ahead/history",
        "params": lambda date, _: {
            "publishTime": date,
            "format": "csv"
        },
        "save_file": "data/dayahead_demand_forecast.csv",
        "chunk_days": 1
    },
    "temperature_data": {
        "api_url": "https://data.elexon.co.uk/bmrs/api/v1/datasets/TEMP",
        "params": lambda start, end: {
            "publishDateTimeFrom": f"{start}T00:00:00Z",
            "publishDateTimeTo": f"{end}T23:59:59Z",
            "format": "csv"
        },
        "save_file": "data/temperature_data.csv",
        "chunk_days": None,  
        "chunk_type": "monthly",
        "headers": {
            "accept": "text/plain"
        }
    },
    "market_index_prices": {
        "api_url": "https://data.elexon.co.uk/bmrs/api/v1/balancing/pricing/market-index",
        "params": lambda start, end: {
            "from": start,
            "to": end,
            "settlementPeriodFrom": 1,
            "settlementPeriodTo": 50,
            "dataProviders": "APXMIDP",
            "format": "csv"
        },
        "save_file": "data/market_index_prices.csv",
        "chunk_days": 7
    }
}

In [101]:
def generic_fetch(name, start_date, end_date):
    config = FETCH_CONFIGS[name]
    chunk_type = config.get("chunk_type", "days")  # default to daily chunking
    chunk_days = config.get("chunk_days")
    headers = config.get("headers", {})

    def fetch_callback(start, end):
        params = config["params"](start, end)
        fetch_csv_data(
            api_url=config["api_url"],
            params=params,
            save_file=config["save_file"],
            headers=headers
        )

    if chunk_type == "monthly":
        current = start_date
        while current <= end_date:
            next_month = (current.replace(day=1) + timedelta(days=32)).replace(day=1) - timedelta(days=1)
            batch_end = min(next_month, end_date)
            fetch_callback(current.strftime('%Y-%m-%d'), batch_end.strftime('%Y-%m-%d'))
            current = batch_end + timedelta(days=1)
    else:
        fetch_in_chunks(start_date, end_date, chunk_days, fetch_callback)


In [102]:
START_DATE = datetime(2022, 10, 1)
END_DATE = datetime(2024, 12, 31)

# for dataset_name in FETCH_CONFIGS:
#     print(f"Fetching: {dataset_name}")
#     generic_fetch(dataset_name, START_DATE, END_DATE)

Possible Reasons for Missing Market Price Data
 
- Looking at the missing periods:
  - 2022-10-29 to 2022-11-04
  - 2023-10-28 to 2023-11-03
  - 2024-10-26 to 2024-11-01
- These dates consistently fall at the end of October and the beginning of November each year.
- Potential Cause: Daylight Saving Time (DST) Changes
    - Many countries, including the UK and parts of Europe, switch from DST to Standard Time on the last weekend of October. This transition could lead to gaps or shifts in settlement period reporting.

# Compiling Master Dataset


In [103]:
expected_start = pd.Timestamp("2022-10-01")
expected_end = pd.Timestamp("2024-12-31")

Market Index Price

In [104]:
df = pd.read_csv("data/market_index_prices.csv")
df['SettlementDate'] = pd.to_datetime(df['SettlementDate'])
df_sorted = df.sort_values(by=['SettlementDate', 'SettlementPeriod'])

# Create a DataFrame of all possible settlement dates and periods
all_dates = pd.date_range(start=expected_start, end=expected_end)
all_periods = range(1, 49) 

# Generate a full DataFrame of all possible combinations
full_index = pd.MultiIndex.from_product([all_dates, all_periods], names=["SettlementDate", "SettlementPeriod"])
full_df = pd.DataFrame(index=full_index).reset_index()

# Merge with the existing data, filling missing values with mean of 3 previous periods
df_filled = full_df.merge(df_sorted, on=["SettlementDate", "SettlementPeriod"], how="left")
df_filled["Price"] = df_filled["Price"].fillna(df_filled["Price"].rolling(3, min_periods=1).mean())
df_filled["Volume"] = df_filled["Volume"].fillna(df_filled["Volume"].rolling(3, min_periods=1).mean())
df_filled["StartTime"] = pd.to_datetime(df_filled["StartTime"], errors="coerce")

price_df = df_filled
price_df.drop(columns=['DataProvider','StartTime'],inplace = True)

Temperature

In [105]:
temperature_df = pd.read_csv("data/temperature_data.csv")
temperature_df["MeasurementDate"] = pd.to_datetime(temperature_df["MeasurementDate"], format="%Y-%m-%d")

# Create a full range of Settlement Periods (1 to 48) for each MeasurementDate
temperature_expanded = temperature_df.copy()
temperature_expanded = temperature_expanded.loc[temperature_expanded.index.repeat(48)]
temperature_expanded["SettlementPeriod"] = list(range(1, 49)) * len(temperature_df)

# Merge with the existing half-hourly market index dataset
df_merged = price_df.merge(
    temperature_expanded,
    left_on=["SettlementDate", "SettlementPeriod"],
    right_on=["MeasurementDate", "SettlementPeriod"],
    how="left"
)
df_merged.drop(columns=["MeasurementDate"], inplace=True)

# Interpolate temperature values across the day for smoother transitions
df_merged["Temperature"] = df_merged.groupby("SettlementDate")["Temperature"].transform(lambda x: x.interpolate())

Wind Forecast Intraday

In [106]:
wind_forecast_df = pd.read_csv("data/wind_generation_forecast.csv")
wind_forecast_df['StartTime'] = pd.to_datetime(wind_forecast_df['StartTime'], utc=True)
wind_forecast_df = wind_forecast_df.drop_duplicates(subset=['StartTime'])

# Create a complete hourly index from 1st Oct 2022 to 31st Dec 2024 in UTC
full_wind_index = pd.date_range(start='2022-10-01 00:00:00', end='2024-12-31 23:30:00', freq='h', tz='UTC')
full_wind_df = pd.DataFrame(full_wind_index, columns=['StartTime'])
full_wind_df = full_wind_df.merge(wind_forecast_df, on='StartTime', how='left')

# Fill missing values for all columns using forward fill, then backfill as a fallback
full_wind_df.ffill(inplace=True)
full_wind_df.bfill(inplace=True)

# Convert hourly data to half-hourly
half_hourly_wind_index = pd.date_range(start='2022-10-01 00:00:00', end='2024-12-31 23:30:00', freq='min', tz='UTC')[::30]
half_hourly_wind_df = pd.DataFrame(half_hourly_wind_index, columns=['StartTime'])
half_hourly_wind_df = half_hourly_wind_df.merge(full_wind_df, on='StartTime', how='left')
half_hourly_wind_df['Generation'] = half_hourly_wind_df['Generation'].interpolate(method='linear')
half_hourly_wind_df[['Dataset', 'PublishTime']] = half_hourly_wind_df[['Dataset', 'PublishTime']].ffill().bfill()
half_hourly_wind_df['SettlementPeriod'] = (half_hourly_wind_df['StartTime'].dt.hour * 2) + (half_hourly_wind_df['StartTime'].dt.minute // 30) + 1
half_hourly_wind_df.sort_values(by='StartTime', inplace=True)

wind_forecast=half_hourly_wind_df.drop(columns=['Dataset','PublishTime'])
# wind_forecast.to_csv('processed_wind_forecast.csv', index=False)

# Merge with the merged dataframe
wind_forecast["SettlementDate"] = pd.to_datetime(wind_forecast["StartTime"].dt.date)
df_merged["SettlementDate"] = pd.to_datetime(df_merged["SettlementDate"])

# Ensure SettlementPeriod is integer in both datasets
df_merged["SettlementPeriod"] = df_merged["SettlementPeriod"].astype(int)
wind_forecast["SettlementPeriod"] = wind_forecast["SettlementPeriod"].astype(int)
wind_forecast.drop(columns=["PublishTime"], inplace=True, errors="ignore")

# Merge datasets on SettlementDate and SettlementPeriod
df_merged = df_merged.merge(
    wind_forecast,
    on=["SettlementDate", "SettlementPeriod"],
    how="left"
)
df_merged["Time"] = df_merged["SettlementPeriod"].apply(
    lambda x: f"{(x-1)//2:02d}:{'30' if x % 2 == 0 else '00'}"
)
df_merged.sort_values(by=["SettlementDate", "SettlementPeriod"], inplace=True)
df_merged.drop(columns=['StartTime','PublishTime'],inplace=True)
df_merged = df_merged.rename(columns={'Generation': 'Wind Forecast IntraDay'})
# df_merged.to_csv('final_merged_dataset.csv',index=False)

Day Ahead Demand Forecast

In [107]:
demand_forecast_df = pd.read_csv("data/dayahead_demand_forecast.csv")
demand_forecast_df["SettlementDate"] = pd.to_datetime(df["SettlementDate"], errors="coerce", dayfirst=True)
demand_forecast_df["SettlementDate"] = pd.to_datetime(demand_forecast_df["SettlementDate"], errors='coerce')

df_merged["SettlementDate"] = pd.to_datetime(df_merged["SettlementDate"], errors='coerce')
full_settlement_periods = df_merged[["SettlementDate", "SettlementPeriod"]].drop_duplicates()

# Merge to identify missing periods in demand_forecast_df
missing_periods = full_settlement_periods.merge(
    demand_forecast_df, on=["SettlementDate", "SettlementPeriod"], how="left")
missing_rows = missing_periods[missing_periods["TransmissionSystemDemand"].isna()].drop(columns=["TransmissionSystemDemand", "NationalDemand"])

# Backfill missing demand values by merging with the existing demand data and using forward fill
demand_forecast_complete = pd.concat([demand_forecast_df, missing_rows]).sort_values(
    ["SettlementDate", "SettlementPeriod"]
)

demand_forecast_complete["TransmissionSystemDemand"] = demand_forecast_complete["TransmissionSystemDemand"].fillna(method="bfill")
demand_forecast_complete["NationalDemand"] = demand_forecast_complete["NationalDemand"].fillna(method="bfill")

# Perform the merge again with the corrected demand forecast data
df_merged = df_merged.merge(
    demand_forecast_complete, on=["SettlementDate", "SettlementPeriod"], how="inner")
df_merged = df_merged.fillna(0)

Wind & Solar Historical

In [108]:
wind_solar_df = pd.read_csv("data/wind_solar_actual_generation.csv")
wind_solar_df["SettlementDate"] = pd.to_datetime(wind_solar_df["SettlementDate"], errors='coerce')

# Aggregate data by SettlementDate and SettlementPeriod
wind_solar_aggregated = wind_solar_df.pivot_table(
    index=["SettlementDate", "SettlementPeriod"],
    columns="PsrType",
    values="Quantity",
    aggfunc="sum"
).reset_index()

# Rename columns for clarity
wind_solar_aggregated.columns.name = None  # Remove hierarchical column name
wind_solar_aggregated = wind_solar_aggregated.rename(columns={
    "Wind Onshore": "Wind_Onshore_Generation",
    "Wind Offshore": "Wind_Offshore_Generation",
    "Solar": "Solar_Generation"
})
wind_solar_aggregated = wind_solar_aggregated.fillna(0)

# Merge with the existing merged dataset
df_merged = df_merged.merge(
    wind_solar_aggregated, on=["SettlementDate", "SettlementPeriod"], how="left"
)
df_merged = df_merged.fillna(0)

NIV Data
  - Loaded manually as the API doesn't work well

In [109]:
niv_df = pd.read_csv("data/NIV.csv")
niv_df["SettlementDate"] = pd.to_datetime(niv_df["SettlementDate"], format="%d-%m-%Y", errors="coerce")

# Identify missing periods compared to the merged dataset
full_settlement_periods = df_merged[["SettlementDate", "SettlementPeriod"]].drop_duplicates()
missing_niv_periods = full_settlement_periods.merge(niv_df, on=["SettlementDate", "SettlementPeriod"], how="left")
missing_niv_rows = missing_niv_periods[missing_niv_periods["NetImbalanceVolume"].isna()].drop(columns=["NetImbalanceVolume"])

# Backfill missing NIV values using forward fill method
niv_complete = pd.concat([niv_df, missing_niv_rows]).sort_values(["SettlementDate", "SettlementPeriod"])
niv_complete["NetImbalanceVolume"] = niv_complete["NetImbalanceVolume"].fillna(method="bfill")

# Merge with the existing merged dataset
df_merged = df_merged.merge(
    niv_complete, on=["SettlementDate", "SettlementPeriod"], how="left"
)
df_merged = df_merged.fillna(0)

Rolling System Demand
  - Loaded manually as the API doesn't work well

In [110]:
rolling_demand_df = pd.read_csv("data/demand_outturn_half_hourly_2022_2024_1.csv")
rolling_demand_df["StartTime"] = pd.to_datetime(rolling_demand_df["StartTime"], errors="coerce")
rolling_demand_df["SettlementDate"] = rolling_demand_df["StartTime"].dt.date
rolling_demand_df["SettlementDate"] = pd.to_datetime(rolling_demand_df["SettlementDate"])

# Ensure that all required settlement periods are present
full_settlement_periods = df_merged[["SettlementDate", "SettlementPeriod"]].drop_duplicates()
missing_demand_periods = full_settlement_periods.merge(
    rolling_demand_df, on=["SettlementDate", "SettlementPeriod"], how="left"
)
missing_demand_rows = missing_demand_periods[missing_demand_periods["Demand"].isna()].drop(columns=["Demand"])
rolling_demand_complete = pd.concat([rolling_demand_df, missing_demand_rows]).sort_values(
    ["SettlementDate", "SettlementPeriod"]
)
rolling_demand_complete["Demand"] = rolling_demand_complete["Demand"].fillna(method="bfill")

# Merge with the existing final dataset
df_merged = df_merged.merge(
    rolling_demand_complete, on=["SettlementDate", "SettlementPeriod"], how="left"
)
df_merged = df_merged.fillna(0)
df_merged = df_merged.rename(columns={'Demand': 'Rolling_System_Demand'})
df_merged.columns = [col.replace(' ', '_') for col in df_merged.columns]
df_merged.drop(columns = ['StartTime_x','StartTime_y'], inplace =True)

FINAL DATA SET

In [111]:

# Drop original SettlementDate and Time columns
df_merged["Datetime"] = pd.to_datetime(df_merged["SettlementDate"].astype(str) + " " + df_merged["Time"])
df_merged.drop(columns=["SettlementDate", "Time"], inplace=True)

# Reorder columns for better readability (Datetime first)
cols = ["Datetime"] + [col for col in df_merged.columns if col != "Datetime"]
df_merged = df_merged[cols]
df_merged.to_csv("data/final_dataset.csv")